# Shelly-Ann Duncan
# 11/19/22
# Project 3 - IMDB - Part 2 

# Import necessary libraries

In [1]:
# imports
import pandas as pd
import numpy as np

import os, json, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [5]:
# load API credentials
with open('/Users/shell/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['API Key'])

In [6]:
# instantiate tmdb API variable
tmdb = login['API Key']

In [7]:
# specify folder for saving data
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok = True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'title.akas (1).tsv.gz',
 'title.basics.tsv.gz',
 'title.ratings (1).tsv.gz',
 'title_akas.csv.gz',
 'title_akas_chunk_001.csv.gz',
 'title_basics.csv.gz',
 'title_basics_cleaned (1).csv.gz',
 'title_ratings.csv.gz']

# Extract budget, revenue and certification from TMDB

In [8]:
# define function to get movie with certification included

def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celioa/tmdbsimple"""
    # get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    
    # loop through coountries in releases
    for c in releases['countries']:
       
        # if the country abbreviation==US
        if c['iso_3166_1'] == 'US':
            
            # save a "certification" key in info with the certification 
            info['certification'] = c['certification']
    
    return info

In [9]:
# define a function to load/write the new json files

def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        
        # Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        
        # Sets file's current position at offset.
        file.seek(0)
        
        # convert back to json.
        json.dump(file_data, file)

In [10]:
# check for requested info
movie = tmdb.Movies(603)

AttributeError: 'str' object has no attribute 'Movies'

In [ ]:
info = movie.info()

In [ ]:
# print the budget info
print(info['budget'])

In [ ]:
# print the revenue info
print(info['revenue'])

In [ ]:
# print the imdb_id info
print(info['imdb_id'])

In [ ]:
test = get_movie_with_rating("tt0848228", "tt0332280") 

test_ids = ["tt0848228","tt0332280"]
results = []
for movie_id in test_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except: 
        pass
test
pd.DataFrame(results)

In [ ]:
# check error list
print(f"- Number of errors: {len(errors)}")
errors

In [ ]:
# movies that started in 2000 or 2001
filter_years[200, 2001]

In [ ]:
# load basics dataset from part 1
basics = pd.read_csv('Data/title_basics_cleaned.csv.gz')
basics.head()

In [ ]:
# create a for loop to filter through the years
# Outer loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc ='YEARS', position=0):
    
    # Define the JSON file to store results for year
    # This will show at the beginng 
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check to see if our file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # Now, create the file if it exists, or leave it if it does exist
    if file_exists == False:
    
        # Save an empty dict to a new json file to append later
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}],f)
        
    # Now, save our specifisc years as the current df YEAR = 2000
    df_2000 = basics.loc[basics['startYear'] == YEARS].copy()
    df_2000.head()

    # Save movie ids from the year 2000 to a list
    movie_ids = df_2000['tconst'].copy()
    movie_ids

    # Check for any previous data
    previous_df = pd.read_json(JSON_FILE)
    previous_df # Will be empty at first
    
    # Filter out any ids that already exist in our JSON_FILE (helps when returning to our problem later)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
# Get index and movie id from list
# Inner loop

    for movie_id in tqdm_notebook(movie_ids_to_get,
                                      desc = f'Movies from {YEAR}',
                                      position = 1,
                                      leave = True):
            # Make attempt to retrieve data for movie id
        try:
            temp = get_movie_with_rating(movie_id) # Our first function

                # Extend results of our file with our other function
            write_json(temp, JSON_FILE)

                # Create a short sleep (helps bog down the server less)
            time.sleep(0.02)

            # If it fails, make a dict with just the id
        except Exception as e:
            continue
            
        # Finally, store our info in the JSON_FILE
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER} final_tmdb_data_{YEARS}.csv.gz
                         compression = 'gzip', index = False)

In [ ]:
# get the final dataframe
final_year_df

In [ ]:
print(f"- Total errors: {len(errors)}")

In [ ]:
# in case of error when trying to run pd.read_json
previous_df = read_and_fix_json(JSON_FILE)